*****************************************************************
#  The Social Web: data representation
- Instructors: Davide Ceolin, Emma Beauxis-Aussalet.
- TAs: Zubaria Inayat, Maxim Sergeev, Zhuofan Mei, Alexander Schmatz, Ling Jin.
- Exercises for Hands-on session 2
*****************************************************************

In this session you are going to mine data in various microformats. You will see the differences in what each of the formats can contain and what purpose they serve. We will start by looking at geographical data.

Prerequisites:
- Python 3.8
- Python packages: requests, BeautifulSoup4, HTMLParser, rdflib


In [9]:
# If you're using a virtualenv, make sure it's activated before running
# this cell!
!pip install requests
!pip install BeautifulSoup4
!pip install HTMLParser
!pip install rdflib
!pip install cloudscraper


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for HTMLParser: filename=HTMLParser-0.0.2-py3-none-any.whl size=5992 sha256=25757c5cec22808c9ea0cc8977c631eeeae62a9dfad3f3b0f531011008dd669f
  Stored in directory: c:\users\radua\appdata\local\pip\cache\wheels\a4\3b\1a\c6cf6093dcb4a243bebf253c48acd3ffa93c8c48685a4fbb05
Successfully built HTMLParser



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/99.7 kB ? eta -:--:--
     --------------- ---------------------- 41.0/99.7 kB 667.8 kB/s eta 0:00:01
     ---------------------------------------- 99.7/99.7 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/54.5 kB ? eta -:--:--
     ---------------------------------------- 54.5/54.5 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


##  Exercise 1

Even if web pages do not use microformat, interesting data can often be extracted from the HTML. You may use packages such as BeautifulSoup to extract arbitrary pieces of data from any HTML page.
The example below shows how we can find the URL of first image in the infobox table of the wikipedia page on Amsterdam. Tip: compare the code below with HTML source code of the wikipedia page: the image url is in the "src" attribute of the "img" element of in the "table" element with class="infobox".

In [ ]:
# -*- coding: utf-8 -*-

import requests
from bs4 import BeautifulSoup

# This script requires you to add a url of a page with geotags to the commandline, e.g.
# python geo.py 'http://en.wikipedia.org/wiki/Amsterdam'
URL = 'https://en.wikipedia.org/wiki/Amsterdam'

req = requests.get(URL, headers={'User-Agent' : "Social Web Course Student"})
soup = BeautifulSoup(req.text)
# print(req.text)
image1 = soup.findAll('table', class_='infobox')[0].find('img')
print(image1['src'])  


Extracting coordinates from a webpage and reformatting them in the geo microformat (based on Example 8-1 in Mining the Social Web). Note that wikipages may encode long/lat information in different ways. On of the ways used by the Amsterdam wikipedia page is in a span element that is not shown to the user: 
<span class="geo">52.367; 4.900</span>
This span element has a single child: len(geoTag == 1) and no further structure, we have to manually get the long/lat by splitting the string on the ';' semicolon.

In [ ]:

geoTag = soup.find(True, 'geo')
print(geoTag)

if geoTag and len(geoTag) > 1:
        lat = geoTag.find(True, 'latitude').string
        lon = geoTag.find(True, 'longitude').string
        print ('Location is at'), lat, lon
elif geoTag and len(geoTag) == 1:
        (lat, lon) = geoTag.string.split(';')
        (lat, lon) = (lat.strip(), lon.strip())
        print (('Location is at'), lat, lon)
else:
        print ('Location not found')


### Task 1

Can you convert the output of Exercise 1 into KML? Here is the KML documentation: https://developers.google.com/kml/documentation/?csw=1 and here you can find a simple example of how it is used: https://renenyffenegger.ch/notes/tools/Google-Earth/kml/index

Visualise the point in Google Maps using the following code example: https://developers.google.com/maps/documentation/javascript/examples/layer-kml-features
You will have to create your own KML file for the custom map layer, and provide a URL to the KML file inside the JavaScript code, which means that you have to upload the file somewhere. You can use a service like http://pastebin.com/ to obtain a URL for your KML file —> paste the code there and request the RAW format URL; use this one in this Task1. If it fails to work you can also use KML viewer websites like https://kmzview.com/.

Is KML a microformat, why (not)?

## Exercise 2 
In order to find information in the web we can use microformats such as [hRecipe](https://microformats.org/wiki/hrecipe) or Schema.org's [Recipe](https://schema.org/Recipe). But first, we'll show you how to find arbitrary tags in a webpage.


### Task 2 
Parsing data for a <sub><sup>veggie</sup></sub> spaghetti alla carbonara recipe (from Example 2-7 in Mining the Social Web).

In [2]:
import cloudscraper
import json
from bs4 import BeautifulSoup

# A yummy webpage (feel free to change to your likings.)
URL = "https://www.acouplecooks.com/spring-vegetarian-spaghetti-carbonara"

# Create a CloudScraper object
scraper = cloudscraper.create_scraper()

# Use the CloudScraper object to fetch the HTML content
response = scraper.get(URL)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Now you can work with the 'soup' object as you did before
listchildren = list(soup.children)
#print(listchildren)


We can find any element in the page through *css tag selectors*
You can find them all [here](https://www.w3schools.com/cssref/css_selectors.asp), but shortly these are "." for classes, # for ids and plain text for the element name.


You can also combine them, so that looking for ".class1.class2" would select all elements displaying both classes. For a deeper overview please check the above link (or google "html tag selectors"). 

In [3]:
print(len(listchildren)) # we can see here how many children the html doc has got.
title_unparsed = soup.select_one("title")
#show the title element
print(title_unparsed)

2
<title>Vegetarian Carbonara – A Couple Cooks</title>


Not so pretty.... Use the text method.

In [4]:
print(title_unparsed.text)

Vegetarian Carbonara – A Couple Cooks


The website has a block of JSON-LD data embedded. Try to see if you can find it in the soup object.
We can load the JSON-LD script to work with it easier.
Lets get a list of the ingredients.

In [5]:
# Find the script tag containing the JSON-LD data
json_ld_script = soup.find("script", {"class": "yoast-schema-graph"})

# Extract the content of the script tag
script_content = json_ld_script.string

# Load the JSON data from the script content
data = json.loads(script_content)

# Access the "recipeIngredient" list
recipe_ingredients = data["@graph"][7]["recipeIngredient"]

# Print the list of ingredients
for ingredient in recipe_ingredients:
    print(ingredient)

1 pound spaghetti noodles
½ cup smoked mozzarella cheese
½ cup grated Parmesan cheese, plus more for serving
4 egg yolks
1 cup frozen Earthbound Farm Organic peas
8 cups Earthbound Farm Organic spinach
3 tablespoons butter
Kosher salt
Fresh ground black pepper


Lets also print out the instructions.

In [6]:
recipe_instructions= data["@graph"][7]["recipeInstructions"]
#the instructions list contains dictionaries as elements, take a look at how the list is organized
for step in recipe_instructions:
    print(step["text"])

In a large pot, combine 6 quarts of water with 2 tablespoons kosher salt and bring it to a boil.
Grate the Parmesan and mozzarella cheese. Carefully separate four egg yolks and set aside.
Once boiling, add the pasta and cook until the pasta is just about al dente, about 7 minutes; then add peas and spinach and cook for 1 minute. Reserve 1 cup cooking water, and then drain the pasta and vegetables.
In a skillet, melt the butter, then stir in the cheeses, ¼ cup pasta water, and ¼ teaspoon kosher salt. Stir in the pasta and vegetables until creamy over low heat, adding more pasta water if necessary (note that the mozzarella will stick together in some places).
To serve, top each pasta serving with a whole egg yolk and additional Parmesan cheese, and stir the yolk into the pasta at the table (if you are uncomfortable serving egg yolks at the table, stir the egg yolks into the pasta in the skillet to heat them through). Serve immediately. (Note that the mozzarella cheese can become gummy th

Websites are going to be structured differently. Look at the following JSON-DL snippet.

In [7]:
json_example = {
    "title": "The anarchist cookbook",
    "recipeInstructions": "<ol class=\"recipeSteps\"><li>Cook the linguine according to the packet instructions. </li><li>Meanwhile, carefully crack the eggs into a small bowl and beat them with a fork. Season with a little black pepper, then stir in the ricotta finely grate in most of the lemon zest. </li><li>When the pasta has 3 minutes left, add the peas. Reserve a little cooking water, then drain the linguine and peas, and return to the pan. </li><li>Stir in the egg mixture and spinach with a wooden spoon – they'll cook gently in the residual heat. Add a little pasta water to loosen, if needed. </li><li>Share between bowls and serve with a green salad.</li></ol>",
    "ingredients": ["a lot of effort", "the right mindset"]
}

recipe_instructions = json_example["recipeInstructions"]
example_soup = BeautifulSoup(recipe_instructions, 'html.parser')

In [8]:
#to get a nice and clean list of the instructions, step by step
#we can use the find method to get the first "ol" element with attribute "class.." and then use find_all to get all list elements in there
#then we can strip the list items to obtain the instructions
list_items = example_soup.find('ol', class_='recipeSteps').find_all('li')
instructions = [item.get_text(strip=True) for item in list_items]
print(instructions)

['Cook the linguine according to the packet instructions.', 'Meanwhile, carefully crack the eggs into a small bowl and beat them with a fork. Season with a little black pepper, then stir in the ricotta finely grate in most of the lemon zest.', 'When the pasta has 3 minutes left, add the peas. Reserve a little cooking water, then drain the linguine and peas, and return to the pan.', "Stir in the egg mixture and spinach with a wooden spoon – they'll cook gently in the residual heat. Add a little pasta water to loosen, if needed.", 'Share between bowls and serve with a green salad.']


## Task 2.1
Now it's your turn. Create a function that can scrape any recipe webpage from the same website (other websites will have different class tags). 

Make sure to:

- return itemized content (e.g. ingredients) in a list. You may want to use a list comprehension here.
- Not all items have been cleaned of their html markdown (see variables ```ingredients``` vs. ```instructions_unparsed```. Make sure to return a list with human readable content (i.e. by using the ```.text``` attribute).


In [24]:
#Here you can see the solution for our example website

URL = "https://www.acouplecooks.com/spring-vegetarian-spaghetti-carbonara"

def parse_website(url):
    # Create a CloudScraper object
    scraper = cloudscraper.create_scraper()

    # Use the CloudScraper object to fetch the HTML content
    response = scraper.get(URL)

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Get the title
    title_unparsed = soup.select_one("title")
    fn = title_unparsed.text
    
    json_ld_script = soup.find("script", {"class": "yoast-schema-graph"})

    # Extract the content of the script tag
    script_content = json_ld_script.string

    # Load the JSON data from the script content
    data = json.loads(script_content)

    # Access the "recipeIngredient" list
    recipe_ingredients = data["@graph"]
    
    ingredients = [ingredient for ingredient in recipe_ingredients]
    
    #Access the instructions
    recipe_instructions= data["@graph"][7]["recipeInstructions"]
    #the instructions list contains dictionaries as elements, take a look at how the list is organized
    instructions = [step["text"] for step in recipe_instructions]

    return {'name': fn,
            'ingredients': ingredients,
            'instructions': instructions,
            }
    
recipe = parse_website(URL)
print (recipe)
        

{'name': 'Vegetarian Carbonara – A Couple Cooks', 'ingredients': [{'@type': 'Article', '@id': 'https://www.acouplecooks.com/spring-vegetarian-spaghetti-carbonara/#article', 'isPartOf': {'@id': 'https://www.acouplecooks.com/spring-vegetarian-spaghetti-carbonara/'}, 'author': {'name': 'Sonja Overhiser', '@id': 'https://www.acouplecooks.com/#/schema/person/d3a3c9869553a70f725f007b4656b09b'}, 'headline': 'Vegetarian Carbonara', 'datePublished': '2018-04-04T08:00:29+00:00', 'dateModified': '2021-12-05T02:15:20+00:00', 'wordCount': 804, 'commentCount': 12, 'publisher': {'@id': 'https://www.acouplecooks.com/#organization'}, 'image': {'@id': 'https://www.acouplecooks.com/spring-vegetarian-spaghetti-carbonara/#primaryimage'}, 'thumbnailUrl': 'https://www.acouplecooks.com/wp-content/uploads/2016/04/Spring-Vegetarian-Carbonara-003.jpg', 'keywords': ['Carbonara', 'Cheese', 'Earth', 'Earth-Saving', 'Easy', 'Mediterranean', 'Mozzarella', 'Parmesan', 'Pasta', 'Peas', 'Quick', 'Spaghetti', 'Spinach', 

In [15]:
# -*- coding: utf-8 -*-

import cloudscraper
import json
from bs4 import BeautifulSoup

# Pass in a URL containing hRecipe, such as
# https://www.jamieoliver.com/recipes/pasta-recipes/veggie-carbonara/

URL = "https://www.jamieoliver.com/recipes/pasta-recipes/veggie-carbonara/"

# Parse out some of the pertinent information for a recipe.
# See http://microformats.org/wiki/hrecipe.

#Solution for jamie oliver
def parse_website(url):
    # Create a CloudScraper object
    scraper = cloudscraper.create_scraper()

    # Use the CloudScraper object to fetch the HTML content
    response = scraper.get(url)

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Get the title
    title_unparsed = soup.select_one("title")
    fn = title_unparsed.text
    
    json_ld_script = soup.find("script", {'type':'application/ld+json'})

    # Extract the content of the script tag
    script_content = json_ld_script.string

    # Load the JSON data from the script content
    data = json.loads(script_content)

    # Access the "recipeIngredient" list
    recipe_ingredients = data["recipeIngredient"]
    ingredients = [ingredient.strip() for ingredient in recipe_ingredients]
    
    #Access the instructions
    recipe_instructions= data["recipeInstructions"]
    list_items = BeautifulSoup(recipe_instructions, 'html.parser').find('ol').find_all('li')
    instructions = [item.get_text(strip=True) for item in list_items]
    
    #the instructions list contains dictionaries as elements, take a look at how the list is organized
    # instructions = [step["text"] for step in recipe_instructions]

    return {
            'name': fn,
            'ingredients': ingredients,
            'instructions': instructions,
            }
    
recipe = parse_website(URL)
print (recipe)

<ol class="recipeSteps"><li>Cook the linguine according to the packet instructions. </li><li>Meanwhile, carefully crack the eggs into a small bowl and beat them with 
a fork. Season with a little black pepper, then stir in the ricotta finely grate in most of the lemon zest. </li><li>When the pasta has 3 minutes left, add the peas. Reserve a little cooking water, then drain the linguine and peas, and return to the pan. </li><li>Stir in the egg mixture and spinach with a wooden spoon – they’ll cook gently in the residual heat. Add a little pasta water to loosen, if needed. </li><li>Share between bowls and serve with a green salad.</li></ol>
[<li>Cook the linguine according to the packet instructions. </li>, <li>Meanwhile, carefully crack the eggs into a small bowl and beat them with 
a fork. Season with a little black pepper, then stir in the ricotta finely grate in most of the lemon zest. </li>, <li>When the pasta has 3 minutes left, add the peas. Reserve a little cooking water, then dr

But How can we get information not only from one website,  but from all? 

The answer: microformats.

But rather than extracting with information manually from the schema.org or hRecipe microformats, we can use a package, ```scrape-schema-recipe``` 

Feel free to experiment with it. 

### Task 2.2
hRecipe is a microformat specifically created for recipes.
Can you for example easily compare different dessert recipe ingredients? For inspiration you can look back at the exercises you did in Hands-on session 1 where you compared different sets of tweets.

In [31]:
from prettytable import PrettyTable
import scrape_schema_recipe
import string
from collections import Counter

# get the top 10 words from the ingredients
def get_words(ingredients):
  words = []
  for t in ingredients:
    for w in t.split():
      if len(w) > 3:
        # from https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
        w = w.translate(str.maketrans('', '', string.punctuation)) 
        words.append(w)

  for item in [words]:
    c = Counter(item)

  return c.most_common()[:10]



url = 'https://www.jamieoliver.com/recipes/butternut-squash-recipes/ultimate-pumpkin-pie/'
recipe_list = scrape_schema_recipe.scrape_url(url, python_objects=True)
recipe1 = recipe_list[0]

url = 'https://www.jamieoliver.com/recipes/fruit-recipes/frozen-sour-cherry-berry-pie/'
recipe_list = scrape_schema_recipe.scrape_url(url, python_objects=True)
recipe2 = recipe_list[0]

words1 = get_words(recipe1['recipeIngredient'])
words2 = get_words(recipe2['recipeIngredient'])

# join the two tuple lists
for i in range(0, len(words1)):
  words1[i] = words1[i] + words2[i]

# draw the table
pt = PrettyTable(field_names=['Recipe1', 'Count1', 'Recipe2', 'Count2'])
[ pt.add_row(r) for r in words1 ]
print(pt)


+----------+--------+----------+--------+
| Recipe1  | Count1 | Recipe2  | Count2 |
+----------+--------+----------+--------+
| teaspoon |   3    |   plus   |   2    |
|  ground  |   3    |  extra   |   2    |
|  large   |   2    |  sugar   |   2    |
| pumpkin  |   2    | vanilla  |   2    |
|  sugar   |   2    | cherries |   2    |
|  double  |   2    |  plain   |   1    |
|  cream   |   2    |  flour   |   1    |
|  serve   |   2    | dusting  |   1    |
|  squash  |   1    |  icing   |   1    |
|   such   |   1    |  orange  |   1    |
+----------+--------+----------+--------+


## Exercise 3

Schema.org is one of the most widely used annotations formats. Schema.org is a multipurpose  template that has been created by a consortium consisting of Yahoo!, Google and Microsoft. It can describe entities, events, products etc. Check out the vocabulary specs on Schema.org.

### Task 3

Parsing schema.org microdata. To parse this data you need to install the rdflib-microdata package, which you have done in one of the previous steps.



In [ ]:
from rdflib import Graph

# Source: https://www.youtube.com/watch?v=sCU214rbRZ0
# Pass in a URL containing Schema.org microformats
URL = "http://dbpedia.org/resource/Micheal_Jackson"

# Initialize a graph
g = Graph()

# Parse in an RDF file graph dbpedia
result = g.parse(location=URL)

# Loop through first 10 triples in the graph
for index, (sub, pred, obj) in enumerate(g):
    print(sub, pred, obj)
    if index == 10:
        break

In [ ]:
# Print the size of the Graph
print(f'Graph has {len(g)} facts')

In [ ]:
# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format='ttl'))

### Task 3.1 
Compare the schema.org information about a band on last.fm to the Facebook Open Graph information about the same band from Facebook. What are the differences? Which format do you think supports better interoperability?

### Task 3.2
Explore the various microformats at http://microformats.org/ and compare the output of the exercises with the output of http://microformats.org/. Think about possible microformats you want to support in your final assignment and read up on how to parse them.